In [13]:
from datasets import load_dataset, get_dataset_config_names, concatenate_datasets, DatasetDict

In [2]:
repo_id = "caitwong/balanced_translation_dataset-finetuning"

In [3]:
config_names = get_dataset_config_names(repo_id)

In [4]:
subsets = [load_dataset(repo_id, config) for config in config_names]

In [5]:
key = "train"
for idx, subset in enumerate(subsets):
    assert subset.keys() == {key} # each subset should only have "train" column

    subsets[idx] = subsets[idx][key].map(lambda x: {col: None if x[col] == "NA" else x[col] for col in subsets[idx][key].column_names})

Map:   0%|          | 0/6034 [00:00<?, ? examples/s]

Map:   0%|          | 0/5037 [00:00<?, ? examples/s]

Map:   0%|          | 0/5074 [00:00<?, ? examples/s]

Map:   0%|          | 0/6896 [00:00<?, ? examples/s]

Map:   0%|          | 0/10589 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5093 [00:00<?, ? examples/s]

Map:   0%|          | 0/20524 [00:00<?, ? examples/s]

Map:   0%|          | 0/8037 [00:00<?, ? examples/s]

Map:   0%|          | 0/17892 [00:00<?, ? examples/s]

Map:   0%|          | 0/7216 [00:00<?, ? examples/s]

In [8]:
combined_dataset = concatenate_datasets(subsets)
print(combined_dataset.column_names)

['idx', 'source_file', 'target_lang', 'en', 'zh', 'id', 'hi', 'ms', 'tl', 'vi', 'th']


In [11]:
columns_to_keep = ["idx", "target_lang", "source_file", "en"]

languages = [col for col in combined_dataset.column_names if col not in columns_to_keep]

assert set(languages) == {'zh', 'id', 'hi', 'ms', 'tl', 'vi', 'th'}

In [14]:
language_datasets = {
    lang: combined_dataset.filter(
        lambda x: x["target_lang"] == lang
    ).map(
        lambda x: {
            **{col: x[col] for col in columns_to_keep},
            "other": x[lang]
        },
        remove_columns=[col for col in combined_dataset.column_names if col not in columns_to_keep]
    )
    for lang in languages
}

Filter:   0%|          | 0/952182 [00:00<?, ? examples/s]

Map:   0%|          | 0/136026 [00:00<?, ? examples/s]

Filter:   0%|          | 0/952182 [00:00<?, ? examples/s]

Map:   0%|          | 0/136026 [00:00<?, ? examples/s]

Filter:   0%|          | 0/952182 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/136026 [00:00<?, ? examples/s]

Filter:   0%|          | 0/952182 [00:00<?, ? examples/s]

Map:   0%|          | 0/136026 [00:00<?, ? examples/s]

Filter:   0%|          | 0/952182 [00:00<?, ? examples/s]

Map:   0%|          | 0/136026 [00:00<?, ? examples/s]

In [15]:
language_datasets = DatasetDict(language_datasets)

In [ ]:
new_repo_id = "keeve101/balanced-multi-corpora-mt"

upload_kwargs = {
    "dataset_name": new_repo_id.split("/")[-1],
    "tags": [
        "shareAI/ShareGPT-Chinese-English-90k",
        "Wikidepia/IndoParaCrawl", # Data leakage, NLLB already used it
        "rhyliieee/tagalog-filipino-english-translation",
        "baudm/tl2en-nmt",
        "cfilt/iitb-english-hindi",
        "vistec-AI/dataset-releases/scb-mt-en-th-2020_v1.0",
        "stefan-it/nmt-en-vi",
        "Corpus-OpenSubtitles"
    ],
    "languages": ['en', 'zh', 'id', 'hi', 'ms', 'tl', 'vi', 'th']
}

for key, dataset in language_datasets.items():
    dataset.push_to_hub(new_repo_id, key)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/137 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/137 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/137 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/137 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

In [23]:
zh_dataset = language_datasets["zh"]

In [26]:
print(zh_dataset[0])

{'idx': 167383, 'source_file': 'computer_en_zh_26k_combined.jsonl', 'target_lang': 'zh', 'en': 'give me 10 problem in  chatgpt', 'other': '给我10个关于chatGPT的问题。'}


In [27]:
config_names = get_dataset_config_names(new_repo_id
)

In [28]:
print(config_names)

['hi', 'id', 'ms', 'th', 'tl', 'vi', 'zh']


In [33]:
dataset = concatenate_datasets([load_dataset(new_repo_id, config_name, split="train") for config_name in config_names])

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=0)
print(train_test_split.column_names)

{'train': ['idx', 'source_file', 'target_lang', 'en', 'other'], 'test': ['idx', 'source_file', 'target_lang', 'en', 'other']}


In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

LANGUAGES = {
   'hi': "hin_Deva", 
   #'id': "ind_Latn", # remove first, data leakage
   'ms': "zsm_Latn",
   'th': "tha_Thai",
   'tl': "tgl_Latn",
   'vi': "vie_Latn",
   'zh': "zho_Hans",
   'en': "eng_Latn"
}

model_path = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_path)

class DataCollatorForSeq2SeqBidirectional(DataCollatorForSeq2Seq):
    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None, padding=True)


c:\Users\keith\anaconda3\envs\llmchat\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

c:\Users\keith\anaconda3\envs\llmchat\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\keith\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [39]:
import torch

class CustomDataCollator(DataCollatorForSeq2Seq):
    def __call__(self, features):
        # Tokenize each example in the batch
        input_ids = []
        labels = []

        for feature in features:
            lang1 = LANGUAGES[feature['target_lang']] 
            lang2 = LANGUAGES["en"]  

            # Set the tokenizer's source language dynamically
            self.tokenizer.src_lang = lang1
            input_batch = self.tokenizer(
                feature['en'], return_tensors='pt', padding=True, truncation=True, max_length=self.max_length
            )

            # Set the tokenizer's target language dynamically
            self.tokenizer.src_lang = lang2
            target_batch = self.tokenizer(
                feature['other'], return_tensors='pt', padding=True, truncation=True, max_length=self.max_length
            )

            input_ids.append(input_batch['input_ids'])
            labels.append(target_batch['input_ids'])

        # Return a dictionary with input_ids and labels for training
        batch = {
            'input_ids': torch.cat(input_ids, dim=0),
            'labels': torch.cat(labels, dim=0)
        }

        return batch

In [40]:
data_collator = CustomDataCollator(tokenizer=tokenizer, padding=True)

data_collator([zh_dataset[0]])

KeyError: 'en'

In [41]:
print(zh_dataset[0])

{'idx': 167383, 'source_file': 'computer_en_zh_26k_combined.jsonl', 'target_lang': 'zh', 'en': 'give me 10 problem in  chatgpt', 'other': '给我10个关于chatGPT的问题。'}
